IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import *
from sklearn.metrics import confusion_matrix, classification_report

print('Libraries imported.')

Libraries imported.


Setting up the environment

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing the dataset through pandas

In [3]:
import json
filename = '/content/drive/My Drive/ML/noduplicatedataset.json'
db = pd.read_json(filename, lines=True)
print('File loaded: %d samples.\n\n' %(len(db.index)))
print(db)

###
# From now on the elements of the json file can b accessed
# as they were attributes of the object db
# e.g. db.semantic[0] will return 'string'
# which is the semantic of the firt tuple of the json file
###

File loaded: 6073 samples.


         id  ...                                                cfg
0       828  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
1     11786  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
2     12621  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
3     11166  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
4     10432  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
...     ...  ...                                                ...
6068  12484  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
6069   7809  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
6070   9806  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
6071    421  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
6072   2008  ...  {'directed': True, 'graph': [], 'nodes': [{'id...

[6073 rows x 4 columns]


Compute vectorizer terms for all instances

In [10]:
# In this way we are able to manipulate the file as a vector of numbers

db.data = db.lista_asm
db.target = db.semantic

vectorizer = CountVectorizer() # multivariate

X_all = vectorizer.fit_transform(db.data)
y_all = db.target
print(vectorizer.get_feature_names())

print(X_all.shape)
print(y_all.shape)

['0x10', '0x100', '0x1000', '0x10000', '0x100000000', '0x1001', '0x1004', '0x1005', '0x10056', '0x1007', '0x1008', '0x1009', '0x100c', '0x100c2', '0x100f', '0x101', '0x1010', '0x1011', '0x1013', '0x1014', '0x1015', '0x1016', '0x1017', '0x1018', '0x1019', '0x101a', '0x101c', '0x101d', '0x101f', '0x102', '0x1020', '0x1021', '0x1022', '0x1024', '0x1025', '0x1026', '0x1027', '0x1028', '0x1029', '0x102c', '0x102d', '0x102f', '0x103', '0x1030', '0x1031', '0x1032', '0x1033', '0x1034', '0x1035', '0x1036', '0x1038', '0x1039', '0x103a', '0x103b', '0x103c', '0x103f', '0x104', '0x1040', '0x1041', '0x1042', '0x104240', '0x104248', '0x104250', '0x104258', '0x104260', '0x104268', '0x10426c', '0x104270', '0x1044', '0x1046', '0x1047', '0x1048', '0x1049', '0x104a', '0x104b', '0x104c', '0x104f', '0x105', '0x1050', '0x1051', '0x1052', '0x1053', '0x1054', '0x1055', '0x1056', '0x1057', '0x1058', '0x105b', '0x105c', '0x105d', '0x105e', '0x106', '0x1060', '0x1061', '0x1062', '0x1064', '0x1068', '0x106a', '0x1

In [ ]:
i = 1
string = db.data[i]

xor = string.count('xor')
shl = string.count('shl')+string.count('shr')+string.count('sal')\
               +string.count('sar')+string.count('shld')+string.count('shrd')
cmp_ = string.count('cmp')
xmm = string.count('xmm')
jmp = string.count('jmp') 
mov = string.count('mov')
l = len(db.data[i])

mappa_parametri = {'xor':xor,'shl':shl,'cmp':cmp_,'xmm':xmm,'jmp':jmp,'mov':mov,'length':l}

print(db.target[i])

for x in mappa_parametri:
  print(x + ': ' + str(mappa_parametri.get(x)) + '\t')



math
xor: 5	
shl: 0	
cmp: 0	
xmm: 40	
jmp: 3	
mov: 8	
length: 819	


Split data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
          test_size=0.333, random_state=117)

print("Size of training set: %d" %X_train.shape[0])
print("Size of test set: %d" %X_test.shape[0])

Size of training set: 4050
Size of test set: 2023


Fit classifier

**Choose model to evaluate:**

In [38]:
from sklearn import tree
from sklearn import svm

classifier_name = 'S'

ClassifierMap = {
    'D':[tree.DecisionTreeClassifier],
    'S': [svm.LinearSVC, 'SVM']
    }

classifier = ClassifierMap[classifier_name][0]()

Fit the model

In [39]:
classifier.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [40]:
acc = classifier.score(X_test, y_test)    
print("Accuracy %.3f" %acc)

Accuracy 0.983


**Evaluation**

In [41]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print('\n')
print(classification_report(y_test, y_pred))

[[355   2   5   0]
 [  0 793   4   2]
 [  4   4 156   8]
 [  0   0   6 684]]


              precision    recall  f1-score   support

  encryption       0.99      0.98      0.98       362
        math       0.99      0.99      0.99       799
        sort       0.91      0.91      0.91       172
      string       0.99      0.99      0.99       690

    accuracy                           0.98      2023
   macro avg       0.97      0.97      0.97      2023
weighted avg       0.98      0.98      0.98      2023



**Prediction**

Import the blindset

In [42]:
# Importing the blindset

filename = '/content/drive/My Drive/ML/nodupblindtest.json'
bs = pd.read_json(filename, lines=True)
print('File loaded: %d samples.\n\n' %(len(db.index)))
print(bs)

File loaded: 6073 samples.


        id  ...                                                cfg
0    10351  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
1    14513  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
2     1767  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
3     9384  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
4     3018  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
..     ...  ...                                                ...
616   9661  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
617   6933  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
618  11954  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
619   5769  ...  {'directed': True, 'graph': [], 'nodes': [{'id...
620   7957  ...  {'directed': True, 'graph': [], 'nodes': [{'id...

[621 rows x 3 columns]


Compute vectorizer terms again

In [43]:
# In this way we are able to manipulate the file as a vector of numbers

i=13
item = np.array(list(bs.lista_asm))
new_x = vectorizer.transform(item)
new_y = classifier.predict(new_x)

print('%s %s' %(item,new_y))

["['jmp qword ptr [rip + 0x203b72]', 'jmp qword ptr [rip + 0x203b6a]', 'jmp qword ptr [rip + 0x203b62]', 'jmp qword ptr [rip + 0x203b52]', 'push rbp', 'push r15', 'push r14', 'push r13', 'push r12', 'push rbx', 'sub rsp, 0x18', 'mov r14, rdx', 'mov rbx, rsi', 'mov ebp, edi', 'mov edi, 0x403504', 'call 0xfffffffffffffe50', 'push rbp', 'push r15', 'push r14', 'push r13', 'push r12', 'push rbx', 'mov eax, edi', 'mov qword ptr [rsp - 0x28], rax', 'cmp dword ptr [rax*4 + 0x604074], 0', 'jle 0x1ca', 'push rbp', 'push r15', 'push r14', 'push r13', 'push r12', 'push rbx', 'sub rsp, 0x148', 'mov ebx, edx', 'mov qword ptr [rsp + 0x20], rsi', 'xorps xmm0, xmm0', 'movaps xmmword ptr [rsp + 0x130], xmm0', 'movaps xmmword ptr [rsp + 0x120], xmm0', 'movaps xmmword ptr [rsp + 0x110], xmm0', 'movaps xmmword ptr [rsp + 0x100], xmm0', 'movaps xmmword ptr [rsp + 0xf0], xmm0', 'movaps xmmword ptr [rsp + 0xe0], xmm0', 'movaps xmmword ptr [rsp + 0xd0], xmm0', 'movaps xmmword ptr [rsp + 0xc0], xmm0', 'movaps 